In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows', 300)  # Show all rows


# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/Data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder, '/content/drive/MyDrive/Real_Trader_EDA_R1/Data')  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['ES 09-24_Minute_240.csv',
 'ES 09-24_Minute_5.csv',
 'NQ 09-24_Minute_240.csv',
 'ES 09-24_Minute_60.csv',
 'NQ 09-24_Minute_5.csv',
 'NQ 09-24_Minute_60.csv',
 'ZB 09-24_Minute_240.csv',
 'ZB 09-24_Minute_5.csv',
 'ZB 09-24_Minute_60.csv',
 '.ipynb_checkpoints',
 'Updated_target.csv',
 'ES_5Min_Actual_Pred.csv',
 'New_5min_data (2).csv',
 'Actual_target.csv',
 'ideal_target.csv']

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Data/New_5min_data (2).csv")
data.drop(columns=["Unnamed: 0"],axis=1,inplace=True)
data['Time Stamp'] = pd.to_datetime(data['Time Stamp'])

In [ ]:
# df = data[['Time Stamp', 'Open', 'High', 'Low', 'Close','BuyersInControlTime', 'BuyersInControlPrice',
#        'SellersInControlTime', 'SellersInControlPrice', 'Entry', 'Defence','Stop', 'Target']]

In [ ]:
# df = data[['Time Stamp', 'Open', 'High', 'Low', 'Close','BuyersInControlTime', 'BuyersInControlPrice',
#        'SellersInControlTime', 'SellersInControlPrice', 'Entry', 'Defence','Stop', 'Target']]
# # Define the date range
# start_time = '2019-06-24 00:00:00'
# end_time = '2019-06-24 23:55:00'
# pd.set_option('display.max_rows', 500)

# # Filter the DataFrame using boolean indexing
# f_df = df[(df['Time Stamp'] > start_time) & (df['Time Stamp'] <= end_time)]
# f_df = f_df.reset_index()
# f_df.drop(columns=["index"],axis=1,inplace=True)

In [ ]:
# if len(f_df) > 91:
#     f1_df = f_df.iloc[91:].reset_index(drop=True)
# else:
#     raise IndexError("DataFrame does not have enough rows.")

In [ ]:
import pandas as pd

def calculate_ideal_values(df):
    # Initialize lists to store ideal_buyer and ideal_seller values
    ideal_buyers = []
    ideal_sellers = []

    # Iterate over the rows of the DataFrame
    for i, row in df.iterrows():
        # Initialize the default values for ideal_buyer and ideal_seller
        ideal_buyer = 0
        ideal_seller = 0

        # Process ideal_buyer if BuyersInControlPrice is not zero
        if row['BuyersInControlPrice'] != 0:
            # Identify future rows
            future_rows = df.iloc[i:]
            # Find future matching rows where Close equals the current Stop value
            future_matching_rows = future_rows[future_rows['Close'] == row['Stop']]

            # If there are matching rows, find the maximum Close value up to the first matching row
            if not future_matching_rows.empty:
                max_future_close = future_rows.loc[:future_matching_rows.index[0], 'Close'].max()
                ideal_buyer = max_future_close

        # Process ideal_seller if SellersInControlPrice is not zero
        if row['SellersInControlPrice'] != 0:
            # Identify future rows
            future_rows = df.iloc[i:]
            # Find future matching rows where Close equals the current Stop value
            future_matching_rows = future_rows[future_rows['Close'] == row['Stop']]

            # If there are matching rows, find the minimum Close value up to the first matching row
            if not future_matching_rows.empty:
                min_future_close = future_rows.loc[:future_matching_rows.index[0], 'Close'].min()
                ideal_seller = min_future_close

        # Append the calculated values to the lists
        ideal_buyers.append(ideal_buyer)
        ideal_sellers.append(ideal_seller)

    # Add the new columns to the DataFrame
    df['ideal_buyer'] = ideal_buyers
    df['ideal_seller'] = ideal_sellers
    df["ideal_target"] = df['ideal_buyer']+ df['ideal_seller']
    df.drop(columns=["ideal_buyer","ideal_seller"],axis=1,inplace=True)
    return df


In [ ]:
# Update the DataFrame with ideal values
data= calculate_ideal_values(data)
data.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Data/ideal_target.csv",index=False)